In [1]:
import json
import random as rd
import os
import wandb

from src.globals import *
from src.register_datasets import register_datasets, register_by_ids
from src.pipeline_runner import run_pipeline
from src.train import do_train
from src.test import do_test

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model

In [2]:

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

logger = setup_logger(output="./log/main.log")
logger.setLevel(0)

# Active Learning

## Random Benchmark
first of all we want to benchmark our models when choosing the data randomly 

In [3]:
register_datasets()

In [ ]:
with open(PATH_TRAIN_FULL_JSON) as file:
    train_dict = json.load(file)
        
image_ids_full = [image["image_id"] for image in DatasetCatalog.get(TRAIN_DATASET_FULL)]





cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TEST = (VALIDATION_DATASET_SLIM,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.WARMUP_ITERS = 0
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.OUTPUT_DIR = "./output/al_tester"
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

model = build_model(cfg)
# initialize weights and biases
wandb.init(project="activeCell-ACDC", sync_tensorboard=True)
result = do_test(cfg, model, logger)
print(result)

rd.seed(0)
sample_ids = rd.sample(image_ids_full, 11) 
missing_ids = list(set(image_ids_full) - set(sample_ids))

results = []


for i in range(7):    
    
    sample_ids += rd.sample(missing_ids, 2)
    print(sample_ids)
    missing_ids = list(set(image_ids_full) - set(sample_ids))
    dataset_name = register_by_ids("test_sample",sample_ids)
    
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.SOLVER.MAX_ITER = len(sample_ids)*20
    
    do_train(cfg, model, logger,resume= i > 0)
    
    result = do_test(cfg, model, logger)
    #result = run_pipeline(cfg)
    print(result)
    results.append(result)
    


WARNING [10/20 13:47:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



wandb: Currently logged in as: florian-bridges. Use `wandb login --relogin` to force relogin


WARNING [10/20 13:47:45 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

WARNING [10/20 13:47:46 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



/home/ubuntu/.local/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


WARNING [10/20 13:47:51 d2.evaluation.coco_evaluation]: No predictions from the model!
OrderedDict([('bbox', {'AP': nan, 'AP50': nan, 'AP75': nan, 'APs': nan, 'APm': nan, 'APl': nan})])
[3156, 3446, 332, 2122, 4189, 3981, 3318, 2485, 3905, 2934, 4780, 1791, 4144]


...

WARNING [10/20 14:18:43 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

WARNING [10/20 14:23:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.790
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.940
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.910
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.093
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.595
 

/home/ubuntu/GitRepos/activeCell-ACDC/src/train.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  wandb.config.update(yaml.load(cfg.dump()))


WARNING [10/20 14:23:34 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

WARNING [10/20 14:28:15 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.813
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.940
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.919
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.813
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.096
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.606
 

/home/ubuntu/GitRepos/activeCell-ACDC/src/train.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  wandb.config.update(yaml.load(cfg.dump()))


WARNING [10/20 14:28:30 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [ ]:
rd.seed(1337)
sample_ids = rd.sample(image_ids_full, 11) 
missing_ids = list(set(image_ids_full) - set(sample_ids))

for i in range(6):
    
    
    sample_ids += rd.sample(missing_ids, 2)
    print(sample_ids)
    missing_ids = list(set(image_ids_full) - set(sample_ids))
    dataset_name = register_by_ids("test_sample",sample_ids)
    
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.SOLVER.MAX_ITER = len(sample_ids)*10
    
    result = run_pipeline(cfg)
    print(result)

In [11]:
result

OrderedDict([('bbox',
              {'AP': 0.0,
               'AP50': 0.0,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': nan,
               'APl': nan}),
             ('segm',
              {'AP': 0.0,
               'AP50': 0.0,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': nan,
               'APl': nan})])

In [14]:
[result['segm']['AP'] for result in results]

[80.98117035873446,
 82.07482402569484,
 82.05156854061143,
 80.98891634243799,
 82.99740969001324,
 82.0667357294776,
 83.68247182744571]